<a href="https://colab.research.google.com/github/YanaAbakumova/Neural_networks/blob/lesson_3/Lesson3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Практическое задание

  1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston. 
  2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

  3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

  1-2. (*)  Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews .Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [3]:
%tensorflow_version 2.x

In [4]:
import tensorflow as tf
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.6.0
TensorFlow 2.0 Hello World


In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
import tensorflow.nn as nn
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [6]:
x = tf.random.uniform([3, 3])

print("есть ли доступные GPU: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("тензор на GPU #0:  "),
print(x.device.endswith('GPU:0'))


есть ли доступные GPU: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
тензор на GPU #0:  
True


In [ ]:
boston = load_boston()
X = pd.DataFrame(boston['data'], columns=boston['feature_names'])
y = boston['target']
                 

In [ ]:
X.shape

(506, 13)

In [17]:
def create_model(X, y, n_layers, n_neurons, n_epochs, activ_f1, activ_f2, loss_func, metrics, bs, val_split, optim):

  level =n_layers
  inputs = Input(shape=(X.shape[1]), name='digits') 
  x=Flatten()(inputs)
  for i in range(level-1):
      x = Dense(n_neurons, name='dense_'+str(i), activation = activ_f1)(x)
  outputs = Dense(1,name='predictions', activation=activ_f2)(x) 

  model = Model(inputs=inputs, outputs=outputs) 

  if optim=='Adam':
    model.compile(optimizer= Adam(learning_rate=0.0001+0.01*i) ,
                        loss=loss_func,
                        metrics=[metrics])
  if optim=='RMSprop':
    model.compile(optimizer= RMSprop(learning_rate=0.0001+0.01*i) ,loss=loss_func,metrics=[metrics])

  hh = model.fit(X, y,epochs=n_epochs, batch_size=bs, validation_split=0.2, verbose = 0)

  return hh.history

In [ ]:
result = pd.DataFrame(columns = ['n_layers', 'n_neurons', 'n_epochs', 'active_func1', 'active_func2','loss_func', 'optim', 'metrics', 'train_result', 'test_result'])

In [ ]:
n_layers = [2, 3, 4]
n_neurons = [32, 64, 128]
n_epochs = [10, 15]
activ_function1 = ['relu', 'sigmoid', 'tanh', 'softmax']
activ_function2 = ['relu', 'sigmoid', 'tanh', 'softmax']
loss_function = ['mse', 'mae']
optim = ['Adam', 'RMSprop']
metrics = ['mse', 'mae']


In [ ]:
for layer in n_layers:
  for neuron in n_neurons:
    for epoch in n_epochs:
      for activ1 in activ_function1:
        for loss in loss_function:
          for optimizer in optim:
            for metric in metrics:
              for activ2 in activ_function2:
                history = create_model(X, y, n_layers = layer, n_neurons = neuron, n_epochs=epoch, activ_f1 = activ1, activ_f2= activ2,loss_func=loss, metrics = metric, bs = 64, val_split=0.2, optim = optimizer)
                result.loc[len(result)] = [layer, neuron, epoch, activ1, activ2, loss, optimizer, metric, history[str(metric)][-1], history['val_'+str(metric)][-1]]



In [ ]:
# Лучшие результаты MAE
result.sort_values('test_result', ascending=True).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
1372,3,128,10,tanh,relu,mae,RMSprop,mae,6.135585,3.633060
1084,3,64,10,sigmoid,relu,mae,RMSprop,mae,6.175420,3.762844
860,3,32,10,tanh,relu,mae,RMSprop,mae,5.907677,3.847974


In [ ]:
# Лучшие результаты MSE
result.loc[result.metrics=='mse'].sort_values('test_result', ascending=True).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
1624,4,32,10,tanh,relu,mae,RMSprop,mse,86.313072,20.090399
1448,3,128,15,sigmoid,relu,mse,RMSprop,mse,72.824570,21.914379
2104,4,128,10,sigmoid,relu,mae,RMSprop,mse,90.246811,22.261532


In [ ]:
# Худшие результаты MSE
result.loc[result.metrics=='mse'].sort_values('test_result', ascending=False).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
400,2,64,15,relu,relu,mae,Adam,mse,17617.568359,39137.128906
280,2,64,10,relu,relu,mae,RMSprop,mse,2798.900635,7802.791504
152,2,32,15,relu,relu,mae,RMSprop,mse,524.919434,4096.118164


In [ ]:
# Худшие результаты MAE
result.loc[result.metrics=='mae'].sort_values('test_result', ascending=False).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
156,2,32,15,relu,relu,mae,RMSprop,mae,86.741982,91.174942
4,2,32,10,relu,relu,mse,Adam,mae,22.959560,28.997257
28,2,32,10,relu,relu,mae,RMSprop,mae,12.867955,28.423422


Улучшить точность нейросети помог подбор кол-ва слоев, нейронов, эпох, комбинации активационных функций, оптимизатора, функции потерь. Использовала 2 метрики: mae и mse. Лучшие результаты достигнуты на 10 эпохах, активационных функциях tanh+relu, с оптимизатором RMSProp. Хороший результат показали модели с 3 слоями и максимальным кол-вом нейронов или 4 слоями и минимальным кол-вом нейронов. Также неплохая комбинация активационных функций - sigmoid + relu. Из функций потерь mae показала резйльтат лучше, чем mse.

Плохие результаты связаны, в основном, с недообученностью модели, всего 2 слоя, мало нейронов. Наихудшая комбинация активационных функций - relu+relu. 

In [26]:
from tensorflow.keras.datasets import imdb
#возьмем 1000 наиболее распространенных слов
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=1000)

In [27]:
train_data.shape

(25000,)

In [39]:
def create_matrix(df):
  length = 1000
  matrix = np.zeros((train_data.shape[0], length))
  for row in range(train_data.shape[0]):
    for num in train_data[row]:
      matrix[row, num] +=1
  matrix /=length
  return matrix

In [40]:
X_train = create_matrix(train_data)

In [41]:
X_train.shape

(25000, 1000)

In [43]:
X_train = X_train.astype('float32')
y_train = np.asarray(train_labels).astype('float32')

In [44]:
y_train.shape

(25000,)

In [46]:
X_train

array([[0.   , 0.001, 0.029, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.041, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.02 , ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.001, 0.055, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.022, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.038, ..., 0.   , 0.   , 0.   ]], dtype=float32)

In [47]:
n_layers = [3, 4]
n_neurons = [64, 128]
n_epochs = [10]
activ_function1 = ['relu','sigmoid', 'tanh']
activ_function2 = ['sigmoid', 'softmax']
loss_function = ['binary_crossentropy', 'Poisson']
optim = ['Adam', 'RMSprop']
metrics = ['accuracy', 'AUC']

In [48]:
result2 = pd.DataFrame(columns = ['n_layers', 'n_neurons', 'n_epochs', 'active_func1', 'active_func2','loss_func', 'optim', 'metrics', 'train_result', 'test_result'])

In [49]:
for layer in n_layers:
  for neuron in n_neurons:
    for epoch in n_epochs:
      for activ1 in activ_function1:
        for loss in loss_function:
          for optimizer in optim:
            for metric in metrics:
              for activ2 in activ_function2:
                history = create_model(X_train, y_train, n_layers = layer, n_neurons = neuron, n_epochs=epoch, activ_f1 = activ1, activ_f2= activ2,loss_func=loss, metrics = metric, bs = 64, val_split=0.2, optim = optimizer)
                result2.loc[len(result2)] = [layer, neuron, epoch, activ1, activ2, loss, optimizer, metric, history[str(metric).lower()][-1], history['val_'+str(metric).lower()][-1]]

In [50]:
# Лучшие результаты Accuracy
result2.loc[result2.metrics=='accuracy'].sort_values('test_result', ascending=False).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
52,3,128,10,relu,sigmoid,binary_crossentropy,RMSprop,accuracy,0.86685,0.8680
16,3,64,10,sigmoid,sigmoid,binary_crossentropy,Adam,accuracy,0.86645,0.8656
132,4,64,10,tanh,sigmoid,binary_crossentropy,RMSprop,accuracy,0.85360,0.8654


In [51]:
# Лучшие результаты AUC
result2.loc[result2.metrics=='AUC'].sort_values('test_result', ascending=False).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
146,4,128,10,relu,sigmoid,binary_crossentropy,Adam,AUC,0.949533,0.939402
150,4,128,10,relu,sigmoid,binary_crossentropy,RMSprop,AUC,0.940472,0.939338
98,4,64,10,relu,sigmoid,binary_crossentropy,Adam,AUC,0.947254,0.939278


In [52]:
# Худшие результаты Accuracy
result2.loc[result2.metrics=='accuracy'].sort_values('test_result', ascending=True).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
93,3,128,10,tanh,softmax,Poisson,RMSprop,accuracy,0.50155,0.4938
129,4,64,10,tanh,softmax,binary_crossentropy,Adam,accuracy,0.50155,0.4938
125,4,64,10,sigmoid,softmax,Poisson,RMSprop,accuracy,0.50155,0.4938


In [53]:
# Худшие результаты AUC
result2.loc[result2.metrics=='AUC'].sort_values('test_result', ascending=True).head(3)

,n_layers,n_neurons,n_epochs,active_func1,active_func2,loss_func,optim,metrics,train_result,test_result
95,3,128,10,tanh,softmax,Poisson,RMSprop,AUC,0.5,0.5
135,4,64,10,tanh,softmax,binary_crossentropy,RMSprop,AUC,0.5,0.5
131,4,64,10,tanh,softmax,binary_crossentropy,Adam,AUC,0.5,0.5


Для этой задачи использованы метрики accuracy, AUC. Лучшие результаты достигнуты c активационными функциями relu+sigmoid, 128 нейронами, лучшая функция потерь binary_crossentropy. Для повышения accuracy больше подошла модель с 3 слоями, оптимизатором RMSProp. Для AUC - модель с 4 слоями и оптимизатором Adam. 

Плохие результаты демонстрирует комбинация активационных функций tanh+softmax.